# Засоби підготовки та налізу даних
## Лабораторна робота №4_1
### ФБ-22 Лаптєв Денис


#### Підготовка середовища

In [1]:
import pandas as pd
import numpy as np
import timeit
import urllib.request
from datetime import time
print ('setup complete')

setup complete


##### Завантаження датасету

In [51]:
def load_data(id = 235):
    url = "https://archive.ics.uci.edu/static/public/"+str(id)+"/data.csv"
    filename = "data/data_1.csv"

    urllib.request.urlretrieve(url, filename)

def create_df():
    df = pd.read_csv("data/data_1.csv")

    df = df.drop(df.loc[df['Sub_metering_1'] =='?'].index)

    df['Global_active_power'] = df['Global_active_power'].astype(float)
    df['Global_reactive_power'] = df['Global_reactive_power'].astype(float)
    df['Voltage'] = df['Voltage'].astype(float)
    df['Global_intensity'] = df['Global_intensity'].astype(float)
    df['Sub_metering_1'] = df['Sub_metering_1'].astype(float)
    df['Sub_metering_2'] = df['Sub_metering_2'].astype(float)
    df['Sub_metering_3'] = df['Sub_metering_3'].astype(float)


    df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))
    df = df.drop(['Date', 'Time'], axis=1)

    return df

def create_arr(df):
    numpy_array = df.values
    # for row in numpy_array:
    #     row[0] = row[0].to_pydatetime().time()
    return numpy_array

#load_data()
df = create_df()
npdf = create_arr(df)

C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


In [96]:
#
type(npdf[0])

numpy.ndarray

In [121]:
df.info()
#df.Time.nunique()
#print(df)
#df.to_csv('output.csv', index=False)
#print(type(numpy_array))
#print(npdf)

<class 'pandas.core.frame.DataFrame'>
Index: 2049280 entries, 0 to 2075258
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   DataTime               datetime64[ns]
 1   Global_active_power    float64       
 2   Global_reactive_power  float64       
 3   Voltage                float64       
 4   Global_intensity       float64       
 5   Sub_metering_1         float64       
 6   Sub_metering_2         float64       
 7   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 140.7 MB


#### Хід роботи

- Виконати всі завдання, використовуючи як numpy array, так і dataframe
- проаналізувати часові витрати на виконання процедур(профілювання часу виконання)
- зробити висновки щодо ситуацій, в яких має сенс віддати перевагу тій чи іншій структурі даних. Висновки оформити звітом із зазначеним часом виконання та оцінкою по 5-бальній шкалі зручності виконання операцій відбору.
- залишити лише ті спостереження, в яких немає порожніх спостережень (порожні значення – пусті поля між роздільником – ? – 28.04.2007, як приклад).

1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [90]:
def fun_1_pd(df):
    result_df = df[(df['Global_active_power'] > 5)]
    return result_df

def fun_1_np(npdf):
    result_npdf = npdf[(npdf[:,1] > 5)]
    return(result_npdf)


SeTuP = '''
from __main__ import fun_1_np, create_df, create_arr, fun_1_pd
df = create_df()
npdf = create_arr(df)'''
_pd= '''
fun_1_pd(df)'''
print("pd:", timeit.repeat(setup=SeTuP, stmt=_pd, repeat=1, number=1))
_np = '''
fun_1_np(npdf)'''
print("np:", timeit.repeat(setup=SeTuP, stmt=_np, repeat=1, number=1))

C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


pd: [0.005864799983100966]


C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


np: [0.04937439999775961]


2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [73]:
def fun_2_pd(df):
    result_df = df[(df['Voltage'] > 235)]
    return result_df

def fun_2_np(npdf):
    result_npdf = npdf[(npdf[:,3] > 235)]
    return result_npdf

SeTuP = '''
from __main__ import fun_2_np, create_df, create_arr, fun_2_pd
df = create_df()
npdf = create_arr(df)'''
_pd= '''
fun_2_pd(df)'''
print("pd:", timeit.repeat(setup=SeTuP, stmt=_pd, repeat=1, number=1))
_np = '''
fun_2_np(npdf)'''
print("np:", timeit.repeat(setup=SeTuP, stmt=_np, repeat=1, number=1))

C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


pd: [0.11963870000909083]


C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


np: [0.39338080000015907]


3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [98]:
def fun_3_pd(df):
    result_df = df[(df['Global_intensity'].between(19, 20)) & (df['Sub_metering_2'] > df['Sub_metering_3'])]
    return result_df

def fun_3_np(npdf):
    result_df = npdf[((npdf[:,4] >= 19) & (npdf[:,4] <= 20) & (npdf[:,6] > npdf[:,7]))]
    return result_df

SeTuP = '''
from __main__ import fun_3_np, create_df, create_arr, fun_3_pd
df = create_df()
npdf = create_arr(df)'''
_pd= '''
fun_3_pd(df)'''
print("pd:", timeit.repeat(setup=SeTuP, stmt=_pd, repeat=1, number=1))
_np = '''
fun_3_np(npdf)'''
print("np:", timeit.repeat(setup=SeTuP, stmt=_np, repeat=1, number=1))

C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


pd: [0.011640500015346333]


C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


np: [0.12289069997495972]


4. Обрати випадковим чином 5000(00)? домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії.

In [101]:
def fun_4_pd(df):
    result_df = df.sample(n=500000)
    avg_1 = result_df['Sub_metering_1'].mean() # mean from pandas
    avg_2 = result_df['Sub_metering_2'].mean()
    avg_3 = result_df['Sub_metering_3'].mean()
    print(f"Sub 1 avg: {avg_1}\nSub 2 avg: {avg_2}\nSub 3 avg: {avg_3}")
    return result_df

def fun_4_np(npdf):
    result_df = npdf[np.random.choice(npdf.shape[0], size=500000)]
    avg_1 = np.mean(result_df[:, 5])  # mean from numpy
    avg_2 = np.mean(result_df[:, 6])
    avg_3 = np.mean(result_df[:, 7])
    print(f"Sub 1 avg: {avg_1}\nSub 2 avg: {avg_2}\nSub 3 avg: {avg_3}")
    return result_df

SeTuP = '''
from __main__ import fun_4_np, create_df, create_arr, fun_4_pd
df = create_df()
npdf = create_arr(df)'''
_pd= '''
fun_4_pd(df)'''
print("pd:", timeit.repeat(setup=SeTuP, stmt=_pd, repeat=1, number=1))
_np = '''
fun_4_np(npdf)'''
print("np:", timeit.repeat(setup=SeTuP, stmt=_np, repeat=1, number=1))

C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


ValueError: a must be 1-dimensional

5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [107]:
x = npdf[1,0]
print(x)
x = x.to_pydatetime()
print(type(x))
print(x.time())


2006-12-16 17:25:00
<class 'datetime.datetime'>
17:25:00


In [111]:

for row in npdf:
    row[0] = row[0].to_pydatetime().time()


In [114]:
def fun_5_pd(df):
    result_df = df[(df["DataTime"].dt.time > time(18, 0)) & (df["Global_active_power"] >= 6)]
    result_df = result_df[(result_df["Sub_metering_2"]>result_df["Sub_metering_1"])&(result_df["Sub_metering_2"]>result_df["Sub_metering_3"])]
    mid = len(result_df) // 2
    first_half = result_df.iloc[:mid:3]
    second_half = result_df.iloc[mid::4]
    result_df = pd.concat([first_half, second_half])
    return result_df

def fun_5_np(npdf):
    result_npdf = npdf
    for row in result_npdf:
        row[0] = row[0].to_pydatetime().time()
    result_npdf = result_npdf[(result_npdf[:,0] > time(18, 0)) & (result_npdf[:,1] >= 6)]

    result_npdf = result_npdf[((result_npdf[:,6] > result_npdf[:,5]) & (result_npdf[:,6] > result_npdf[:,7]))]

    half = len(result_npdf) // 2
    first_half = result_npdf[:half:3]
    second_half = result_npdf[half::4]
    result_npdf = np.concatenate((first_half, second_half))
    return result_npdf

SeTuP = '''
from __main__ import fun_5_np, create_df, create_arr, fun_5_pd
df = create_df()
npdf = create_arr(df)'''
_pd= '''
fun_5_pd(df)'''
print("pd:", timeit.repeat(setup=SeTuP, stmt=_pd, repeat=1, number=1))
_np = '''
fun_5_np(npdf)'''
print("np:", timeit.repeat(setup=SeTuP, stmt=_np, repeat=1, number=1))

C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


pd: [0.6082810999942012]


C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data_1.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_10060\20275659.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


np: [1.8839575999882072]


Не зрозуміло, що краще використовувати: pd чи np. Щоразу швидшим виявляється, то один методі, то інший. Важко прогледіти якісь тенденцію у часі форматування датасету.